<a href="https://colab.research.google.com/github/kozyreviva/Climat_DS/blob/main/Air_indicators/change_air_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Change air indicators

Атмосферный воздух - важная часть окружающей среды. Развитие промышленности, транспорта, увеличение городов, и освоение атмосферного космоса приводот к изменению состава атмосферы и нарушению её естественного баланса.

В проекте рассмотрим некоторые вещества черезмерныей избыток которых ухудшает состав атмосферы:
 * N2O(Оксид азота(I))
 * SO2(Оксид серы(IV))
 * CO2(Оксид углерода(IV))
 * NOx(Оксиды азота)
 * CH4(Метан)
 * Greenhouse Gas(Парниковые газы)

Источниками этих газов в атмосфере земли являются вулканические выбросы, жизнедеятельность биосферы и антропогенные факторы.
Наша задача рассмотреть содержание этих веществ в мире в промежутке с 1990 по 2018 года. Показать процент, на сколько увеличилось содержание газов в странах.

#### Install libraries

In [ ]:
#install libraries
!pip install dataprep
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.0/764.0 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 65.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.4 MB/s eta 0:00:00
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.24-cp310-cp310-linux_x86_64.whl size=1252699 sha256=590990e83

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from dataprep.clean import clean_country

### Functions

In [ ]:
# function for change columns name
def new_col_name(data):
   colnames = list(data.iloc[0])
   colnames[0] = 'country_id'
   colnames[1] = 'country'
   colnames[-3:] = data.columns[-3:]
   data.columns = colnames
   return  data.drop(0).reset_index(drop = True)


In [ ]:
# function for change type
def change_type(data):
    data[data.columns[2:31]] = data[data.columns[2:31]].replace(to_replace = ',', value = '', regex = True)
    data[data.columns[2:31]] = data[data.columns[2:31]].astype(float)
    data[data.columns[31]] = data[data.columns[31]].astype('Int64')
    data[data.columns[32]] = data[data.columns[32]].replace(to_replace = ',', value = '', regex = True)
    data[data.columns[32]] = data[data.columns[32]].astype(float)
    data[data.columns[33]] = data[data.columns[33]].replace(to_replace = ',', value = '', regex = True)
    data[data.columns[33]] = data[data.columns[33]].astype(float)

In [ ]:
# function for creating new data
def country_change_alpha(data):
    new_data = (data[['country','% change since 1990']]
                .sort_values('% change since 1990',ascending = False)
                .reset_index(drop = True)
                .head(10)
                .copy())
    new_data = clean_country(new_data, "country", output_format = 'alpha-3')
    new_data = new_data.rename(columns = {'country_clean':'iso_alpha'})
    return new_data



In [ ]:
# function for creating new data
def change_data_by_years(data):
    lst =[]
    for i, row in df_n2o.iloc[:,2:31].items():
        for item in row:
            lst.append([i,item])
    df = pd.DataFrame(lst)
    df = df.rename(columns = {0:'year',1:'values'})

    new_n2o_country = data['country'].copy()
    for i in range(28):
       new_n2o_country = pd.concat([new_n2o_country,data['country']], ignore_index = True)

    new_df = df.join(new_n2o_country)
    new_df = clean_country(new_df, "country", output_format = 'alpha-3')
    new_df = new_df.rename(columns = {'country_clean':'iso_alpha'})
    return new_df

### Data preprocessing

In [ ]:
# read and save datasets
df_n2o = pd.read_csv("N2O_Emissions.csv")
df_n2o.replace(['...','…'], np.NaN,inplace = True)
df_co2 = pd.read_csv("CO2_Emissions.csv")
df_co2.replace(['...','…'], np.NaN,inplace = True)
df_so2 = pd.read_csv("SO2_emissions.csv")
df_so2.replace(['...','…'], np.NaN,inplace = True)
df_ch4 = pd.read_csv("CH4_Emissions.csv")
df_ch4.replace(['...','…'], np.NaN,inplace = True)
df_ghg = pd.read_csv("GHG_Emissions.csv")
df_ghg.replace(['...','…'], np.NaN,inplace = True)
df_nox = pd.read_csv("NOx_Emissions.csv")
df_nox.replace(['...','…'], np.NaN,inplace = True)

In [ ]:
# show data
df_n2o.head()

,Country ID,Country,"Time series - Total N2O emissions, in 1000 tonnes of CO2 equivalent",Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,"N2O emissions, latest year",% change since 1990,"N2O emissions \nper capita, \nlatest year"
0,NaN,NaN,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,Latest Year,1000 tonnes of \nCO2 equivalent,%,kg
1,4.0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"19,610.00",NaN,NaN,NaN,NaN,NaN,2013,"19,610.00",NaN,0.608
2,8.0,Albania,96.10,136.40,108.50,96.10,96.10,93.00,83.70,77.50,...,NaN,NaN,NaN,NaN,NaN,NaN,2009,120.96,25.87,0.041
3,12.0,Algeria,NaN,NaN,NaN,NaN,"9,300.00",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2000,"6,500.70",NaN,0.209
4,24.0,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2005,"13,869.40",NaN,0.714


In [ ]:
# raname columns
df_n2o = new_col_name(df_n2o)
df_co2 = new_col_name(df_co2)
df_ghg = new_col_name(df_ghg)
df_so2 = new_col_name(df_so2)
df_nox = new_col_name(df_nox)
df_ch4 = new_col_name(df_ch4)

In [ ]:
# change type
change_type(df_n2o)
change_type(df_co2)
change_type(df_ghg)
change_type(df_so2)
change_type(df_nox)
change_type(df_ch4)

In [ ]:
# creating new datasets
n2o_country_change = country_change_alpha(df_n2o)
co2_country_change = country_change_alpha(df_co2)
ghg_country_change = country_change_alpha(df_ghg)
so2_country_change = country_change_alpha(df_so2)
nox_country_change = country_change_alpha(df_nox)
ch4_country_change = country_change_alpha(df_ch4)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
                                     

Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


In [ ]:
# creating new datasets
n2o_data_by_years = change_data_by_years(df_n2o)
so2_data_by_years = change_data_by_years(df_so2)
co2_data_by_years = change_data_by_years(df_co2)
ghg_data_by_years = change_data_by_years(df_ghg)
nox_data_by_years = change_data_by_years(df_nox)
ch4_data_by_years = change_data_by_years(df_ch4)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	5452 values cleaned (100.0%)
Result contains 5452 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	4118 values cleaned (75.53%)
Result contains 4118 (75.53%) values in the correct format and 1334 null values (24.47%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	5452 values cleaned (100.0%)
Result contains 5452 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	4988 values cleaned (91.49%)
Result contains 4988 (91.49%) values in the correct format and 464 null values (8.51%)


Country Cleaning Report:
	5452 values cleaned (100.0%)
Result contains 5452 (100.0%) values in the correct format and 0 null values (0.0%)


In [ ]:
# creating dictionaries
percent_change_dict = {"N2O":n2o_country_change,
        "SO2":so2_country_change,
        "CO2":co2_country_change,
        "GHG":ghg_country_change,
        "NOX":nox_country_change,
        "CH4":ch4_country_change}

values_by_years_dict = {"N2O":n2o_data_by_years,
        "SO2":so2_data_by_years,
        "CO2":co2_data_by_years,
        "GHG":ghg_data_by_years,
        "NOX":nox_data_by_years,
        "CH4":ch4_data_by_years}

### Visualization

In [ ]:
# creating Dash for visualization
app = Dash(__name__)


left_div = (html.Div(dcc.Graph(id="bar")))
right_div = (html.Div(dcc.Graph(id="scatter")))

upper_div = html.Div([html.H3("Percentage change by country", style = {"textAlign":"center"}),
                      html.Div([left_div, right_div], style={"display": "flex"})])
central_div = html.Div([html.H3("Indicators by countries",style={'textAlign': 'center'}),
                        html.Div(dcc.Graph(id = "choropleth"),style={"display": "flex", "justify-content": "center"})])

app.layout = html.Div([
    html.H2("Change in air indicators from 1990 to 2018.", style = {"textAlign":"center"}),
    html.H3("Select indicator:"),
    dcc.RadioItems(
        id = "indicator",
        options = ["N2O", "SO2", "CO2","GHG","NOX","CH4"],
        value = "N2O",
        inline = True),
    html.Div([upper_div,central_div])
    ])

@app.callback(
    Output("bar","figure"),
    Input("indicator","value")
)

def to_bar(indicator):
    bar_fig = px.bar(percent_change_dict.get(indicator),
                     x = "country",
                     y = "% change since 1990")
    return bar_fig


@app.callback(
    Output("scatter","figure"),
    Input("indicator","value")
)

def percents_map(indicator):
    percent_map_fig = px.scatter_geo(percent_change_dict.get(indicator),
                         locations = "iso_alpha",
                         size = "% change since 1990",
                         projection="natural earth",
                         color= "country")
    return percent_map_fig

@app.callback(
    Output("choropleth","figure"),
    Input("indicator","value")
)
def values_to_map(indicator):
    values_to_map_fig = px.choropleth(values_by_years_dict.get(indicator),
                        locations="iso_alpha",
                        color="values",
                        hover_name="country",
                        animation_frame="year"
                        )
    values_to_map_fig.update_layout(
                      margin=dict(l=0, r=0, t=0, b=0),
                      geo=dict(showframe = False,
                               showcoastlines = False
                               )
                      )
    return values_to_map_fig

app.run_server()

<IPython.core.display.Javascript object>